In [1]:
import pandas as pd
import numpy as np
import pickle
import shap
from typing import List, Callable, Optional, Tuple, Any
from sklearn.model_selection import BaseCrossValidator
from sklearn.base import BaseEstimator
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_percentage_error
from pandas.tseries.offsets import MonthEnd

class DateTimeSeriesSplit:
    def __init__(self, n_splits: int = 4, test_size: int = 1, margin: int = 1, window: int = 3):
        self.n_splits = n_splits
        self.test_size = test_size
        self.margin = margin
        self.window = window

    def get_n_splits(self) -> int:
        return self.n_splits

    def split(self, X: pd.DataFrame, y: Optional[Any] = None, groups: pd.DataFrame = None) -> Tuple[np.ndarray, np.ndarray]:
        unique_dates = sorted(groups.unique())
        rank_dates = {date:rank for rank, date in enumerate(unique_dates)}
        X['index_time'] = groups.map(rank_dates)
        X = X.reset_index(drop = True)
        index_time_list = list(rank_dates.values())

        for i in reversed(range(1, self.n_splits + 1)):
            left_train = int((index_time_list[-1] - i*self.test_size + 1 - self.window - self.margin)*(self.window/np.max([1,self.window])))
            right_train = index_time_list[-1] - i*self.test_size - self.margin + 1
            left_test = index_time_list[-1] - i*self.test_size + 1
            right_test = index_time_list[-1] - (i-1)*self.test_size + 1
            index_test = X.index.get_indexer(X.index[X.index_time.isin(index_time_list[left_test: right_test])])
            index_train = X.index.get_indexer(X.index[X.index_time.isin(index_time_list[left_train: right_train])])
            yield index_train, index_test

class Kraken:
    def __init__(self, estimator: BaseEstimator, cv: BaseCrossValidator, metric: Callable, meta_info_name: str):
        self.estimator = estimator
        self.cv = cv
        self.metric = metric
        self.meta_info_name = meta_info_name

    def get_rank_dict(self, X: np.ndarray, y: np.ndarray, list_of_vars: List[str], group_dt: Optional[np.ndarray]):
        self.dict_fold_importances = {'Feature': list_of_vars, 'abs_shap': np.zeros(len(list_of_vars))}
        for fold, (train_idx, val_idx) in enumerate(self.cv.split(X, groups = group_dt), 1):
            X_train, X_test = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[val_idx]
            self.estimator.fit(X_train[list_of_vars], y_train.values)
            explainer = shap.Explainer(self.estimator)
            shap_values = explainer.shap_values(X_test[list_of_vars])
            self.dict_fold_importances['abs_shap'] += np.abs(shap_values).mean(axis=0)
        self.fe_dict = {key: value for key, value in zip(self.dict_fold_importances['Feature'], self.dict_fold_importances['abs_shap'])}
        self.rank_dict = {key: rank for rank, key in enumerate(sorted(self.fe_dict, key=self.fe_dict.get, reverse=True), 1)}

    def get_cross_val_score(self, X: np.ndarray, y: np.ndarray, var: str, old_scores: np.ndarray, selected_vars: Optional[List[str]] = None, group_dt: Optional[np.ndarray] = None, round_num: int = 3):
        if selected_vars is None:
            selected_vars = []
        selected_vars.append(var)
        list_scores = []
        for fold, (train_idx, val_idx) in enumerate(self.cv.split(X, groups=group_dt), 1):
            X_train, X_test = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[val_idx]
            self.estimator.fit(X_train[selected_vars], y_train)
            error = round(self.metric(np.exp(y_test), np.exp(self.estimator.predict(X_test[selected_vars]))), round_num)
            list_scores.append(error)
        fold_scores = np.array(list_scores)
        summa = sum(fold_scores - old_scores < 0) * 1 + sum(fold_scores - old_scores > 0) * -1
        mean_cv_score = round(np.mean(fold_scores), round_num)
        return fold_scores, summa, mean_cv_score

    def get_vars(self, X: np.ndarray, y: np.ndarray, early_stopping_rounds: int = 30, summa_approve: int = 1, best_mean_cv: int = 100, vars_in_model: Optional[List] = list(), group_dt: Optional[np.ndarray] = None, round_num: int = 3, old_scores: Optional[np.ndarray] = None):
        self.round_num = round_num
        if old_scores == None:
            old_scores = np.array([100 for i in range(self.cv.get_n_splits())])
        iteration_step = 0
        the_list_from_which_we_take_vars = [i for i in list(self.rank_dict.keys()) if i not in vars_in_model]
        feature_was_added = True
        while feature_was_added:
            iteration_step = 0
            var_for_add = ''
            print('начинаем след этап', best_mean_cv)
            best_positive_groups = summa_approve
            for var in the_list_from_which_we_take_vars:
                iteration_step += 1
                if iteration_step > early_stopping_rounds:
                    print(f'early_stopping_rounds {early_stopping_rounds}')
                    break
                fold_scores, summa, mean_cv_score = self.get_cross_val_score(X = X, y = y, var = var, old_scores = old_scores, selected_vars = vars_in_model.copy(), group_dt = group_dt, round_num = self.round_num)
                if (summa > best_positive_groups) or (summa == best_positive_groups and mean_cv_score < best_mean_cv):
                    best_positive_groups = summa
                    best_mean_cv = mean_cv_score
                    old_scores = fold_scores
                    var_for_add = var
                    iteration_step = 0
                    print(f'new var_for_add ! {var_for_add}')
            if var_for_add != '':
                vars_in_model.append(var_for_add)
                the_list_from_which_we_take_vars.remove(var_for_add)
                print('едем дальше')
                print('в итоге получили список', vars_in_model)
                list_meta = ['vars_list'] + [best_positive_groups] + [best_mean_cv] + old_scores.tolist()
                df_meta = pd.DataFrame(list_meta).T
                df_meta.columns = ['vars', 'summa', 'mean_cv_scores'] + ['cv' + str(i) for i in range(1, self.cv.get_n_splits() + 1)]
                df_meta.at[0, 'vars'] = vars_in_model.copy()
                try:
                    df_meta_info = pd.concat([df_meta_info, df_meta])
                except:
                    df_meta_info = df_meta.copy()
                df_meta_info.to_csv(f'df_meta_info_{self.meta_info_name}.csv')
                continue
            else:
                feature_was_added = False
        print('мы сошлись')
        print(vars_in_model)
        print(best_mean_cv)
        return vars_in_model

class AddTrain:
    """
    Class for add train
    """
    def __init__(self, df_: pd.DataFrame, model_path: 'str', train_end: str, oot_dates: List[str], vsp_test: np.array, used_features: List[str]):
        """
        Initialize AddTrain class with given df_, model, train_end, oot_dates.
        Args:
            df_ (pd.DataFrame): dataset with all features and target
            model_path (str): old model path from oper plan
            train_end (str): last report date in train set from develop process
            oot_dates (str): list oot dates in df_ from develop process
            vsp_test (np.array): set of test(oos) urf_code_map
            used_features (List): old model has incorrect naming features, that`s why need write explicit
        """
        self.df_ = df_
        self.model_path = model_path
        self.train_end = train_end
        self.oot_dates = oot_dates
        self.vsp_test = vsp_test
        self.used_features = used_features

    def scoring_constant_model(self):
        with open(self.model_path, 'rb') as mod_pkl:
            model = pickle.load(mod_pkl)

        cond1_oot = (self.df_['dt'] > self.train_end)
        X_oot = self.df_[cond1_oot]
        y_oot = np.log(X_oot['target'])

        macro_list = []
        print(X_oot['dt'].value_counts().sort_index())

        for dt, subset in X_oot.groupby('dt'):
            y_pred_oot = np.exp(model.predict(subset[self.used_features]))
            mape_oot = round(mean_absolute_percentage_error(subset['target'], y_pred_oot), 2)
            macro_oot = round(y_pred_oot.sum(), 2)
            macro_fact = subset['target'].sum()
            ape_macro = round(100*(macro_oot - macro_fact)/macro_fact, 2)

            macro_list.append([dt, mape_oot, macro_oot, macro_fact, ape_macro])

        self.results_scor_constant = pd.DataFrame(macro_list, columns = ['dt', 'const_mape_oot', 'const_macro_oot',
                                                                         'const_macro_fact', 'const_ape_macro'])
        #self.results_scor_constant['const_mape_oot'] = self.results_scor_constant['const_mape_oot'] * (-1)

    def scoring_update_model(self):
        with open(self.model_path, 'rb') as mod_pkl:
            old_model = pickle.load(mod_pkl)

        macro_list = []

        for i, _ in enumerate(sorted(self.df_[self.df_['dt'] > self.train_end]['dt'].unique()[0:-3]), 1):
            if i == 1:
                cond1_train = (self.df_['dt'] <= pd.to_datetime(self.train_end) + MonthEnd(len(self.oot_dates)))
                cond2_train = (~self.df_['urf_code_map'].isin(self.vsp_test))
                X_train = self.df_[cond1_train & cond2_train]
                y_train = np.log(X_train['target'])

                cond1_test = (self.df_['urf_code_map'].isin(self.vsp_test))
                X_test = self.df_[cond1_train & cond1_test]
                y_test = np.log(X_test['target'])

                cond1_oot = (self.df_['dt'] == pd.to_datetime(self.train_end) + MonthEnd(len(self.oot_dates) + 2))
                X_oot = self.df_[cond1_oot]
                y_oot = np.log(X_oot['target'])

            elif i > 1:
                cond1_train = (self.df_['dt'] <= pd.to_datetime(self.train_end) + MonthEnd(len(self.oot_dates) + i - 1))
                cond2_train = (~self.df_['urf_code_map'].isin(self.vsp_test))
                X_train = self.df_[cond1_train & cond2_train]
                y_train = np.log(X_train['target'])

                cond1_test = (self.df_['urf_code_map'].isin(self.vsp_test))
                X_test = self.df_[cond1_train & cond1_test]
                y_test = np.log(X_test['target'])

                cond1_oot = (self.df_['dt'] == pd.to_datetime(self.train_end) + MonthEnd(len(self.oot_dates) + i + 1))
                X_oot = self.df_[cond1_oot]
                y_oot = np.log(X_oot['target'])

            print('*-*-*-*-*-*-*-*-*-*- start split train/test/oot *-*-*-*-*-*-*-*-*-*-')
            print(f'step_{i}')

            train_test_vc = pd.merge(X_train['dt'].value_counts().sort_index().reset_index(), X_test['dt'].value_counts().sort_index().reset_index(),  how = 'outer', on = 'index')
            stats_val_cnt = pd.merge(train_test_vc, X_oot['dt'].value_counts().sort_index().reset_index(), how = 'outer', on = 'index')
            stats_val_cnt.columns = ['dt', 'cnt_train', 'cnt_oos', 'cnt_oot']
            display(stats_val_cnt)

            model = LGBMRegressor(**old_model.get_params())

            model.fit(X_train[self.used_features], y_train)

            dt = X_oot['dt'].unique()[0]
            y_pred_oot = np.exp(model.predict(X_oot[self.used_features]))
            mape_oot = round(mean_absolute_percentage_error(X_oot['target'], y_pred_oot), 2)
            macro_oot = round(y_pred_oot.sum(), 2)
            macro_fact = X_oot['target'].sum()
            ape_macro = round(100*(macro_oot - macro_fact)/macro_fact, 2)

            macro_list.append([dt, mape_oot, macro_oot, macro_fact, ape_macro])

        self.results_scor_update = pd.DataFrame(macro_list, columns = ['dt', 'update_w_mape_oot', 'update_w_macro_oot',
                                                                     'update_w_macro_fact', 'update_w_ape_macro'])
        self.results_scor_update['update_w_mape_oot'] = self.results_scor_update['update_w_mape_oot'] * (-1)

    def final_report(self):
        report = pd.merge(self.results_scor_constant,
                         self.results_scor_update,
                         how = 'left',
                         on = 'dt')
        report['diff'] = round(100*(report['update_w_mape_oot'] - report['const_mape_oot'])/report['const_mape_oot'], 2)
        return report
    
    def scoring_new_model(self, start_month: str, window: int, n_splits: int, test_size: int, margin: int, lgbm_params: dict, early_stopping_rounds: int, round_num: int, metric: Callable):
        """
        Метод создает новую модель для каждой даты отчета и эмулирует скоринг с добавлением обучения.

        Параметры:
        window (int): размер окна для DateTimeSeriesSplit.
        n_splits (int): количество разбиений в DateTimeSeriesSplit.
        test_size (int): размер тестовой выборки в DateTimeSeriesSplit.
        margin (int): маржа между тренировочным и тестовым набором в DateTimeSeriesSplit.
        lgbm_params (dict): параметры для инициализации LGBMRegressor.
        early_stopping_rounds (int): количество раундов для ранней остановки в Kraken.
        round_num (int): количество знаков после запятой для округления результатов.
        metric (Callable): метрика для оценки модели (например, mean_absolute_percentage_error).
        """
            # Загрузка старой модели
        with open(self.model_path, 'rb') as file:
            old_model = pickle.load(file)

        start_month_dt = pd.to_datetime(start_month)
        results = []
        meta_info = []

        print(f"Начинаем обработку данных, начиная с {start_month_dt.strftime('%Y-%m')}")

        while start_month_dt <= self.df_['dt'].max():
            print(f"Обрабатываем месяц {start_month_dt.strftime('%Y-%m')}")

            # Разделение на train и OOT
            train_data = self.df_[self.df_['dt'] < start_month_dt]
            oot_data = self.df_[self.df_['dt'] >= start_month_dt]

            # Инициализация DateTimeSeriesSplit и Kraken
            cv_datetime = DateTimeSeriesSplit(window=window, n_splits=n_splits, test_size=test_size, margin=margin) 
            group_dt = train_data['dt']
            model = LGBMRegressor(**lgbm_params)  # Необходимо инициализировать с параметрами
            selector = Kraken(model, cv_datetime, metric, 'updated_model')  # Необходимо инициализировать с параметрами

            # Подбор фичей на основе SHAP значений
            selector.get_rank_dict(train_data, np.log(train_data['target']), self.used_features, group_dt=train_data['dt'])
            new_vars_class = selector.get_vars(train_data, np.log(train_data['target']), vars_in_model=[], early_stopping_rounds=early_stopping_rounds, group_dt=train_data['dt'], round_num=round_num)

            # Обучение новой модели с отобранными переменными
            model.fit(train_data[new_vars_class], np.log(train_data['target']))

            # Оценка новой модели на OOT данных
            y_pred_new = np.exp(model.predict(oot_data[new_vars_class]))
            mape_new = mean_absolute_percentage_error(oot_data['target'], y_pred_new)

            # Оценка старой модели на OOT данных
            y_pred_old = np.exp(old_model.predict(oot_data[self.used_features]))
            mape_old = mean_absolute_percentage_error(oot_data['target'], y_pred_old)

            # Сравнение старой и новой модели
            if mape_new < mape_old:
                print(f"Новая модель ({mape_new}) лучше старой ({mape_old}) для {start_month_dt.strftime('%Y-%m')}")
                old_model = model
                # Сохраняем новую модель
                with open(self.model_path, 'wb') as file:
                    pickle.dump(model, file)
                results.append({'month': start_month_dt.strftime('%Y-%m'), 'model': 'new', 'mape': mape_new})
            else:
                print(f"Старая модель ({mape_old}) лучше новой ({mape_new}) для {start_month_dt.strftime('%Y-%m')}")
                results.append({'month': start_month_dt.strftime('%Y-%m'), 'model': 'old', 'mape': mape_old})

            # Сохраняем метаинформацию
            meta_info.append({
                'month': start_month_dt.strftime('%Y-%m'),
                'features': new_vars_class,
                'mape_new': mape_new,
                'mape_old': mape_old
            })

            start_month_dt += pd.DateOffset(months=1)

        # Сохранение метаинформации в CSV
        meta_info_df = pd.DataFrame(meta_info)
        meta_info_df.to_csv('meta_info.csv', index=False)

        return pd.DataFrame(results)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
import pandas as pd
import numpy as np
import random

# Генерация данных для 10 магазинов за 2 года
np.random.seed(0)
dates = pd.date_range(start='2022-01-01', end='2023-12-31', freq='M')
stores = np.arange(1, 10001)

# Создание DataFrame
data = pd.DataFrame({
    'date': np.repeat(dates, len(stores)),
    'store_id': np.tile(stores, len(dates)),
    'sales': np.random.randint(100, 500, size=len(dates) * len(stores)),
    'other_feature1': np.random.rand(len(dates) * len(stores)),
    'other_feature2': np.random.rand(len(dates) * len(stores)),
    'other_feature3': np.random.rand(len(dates) * len(stores))**2,
    'other_feature4': np.random.rand(len(dates) * len(stores))**3
})

# Добавление примерного целевого признака
data['target'] = data['sales'] * (1 + data['other_feature1'] - 0.5*data['other_feature2'])*(data.date.dt.year - 2020)*(data.date.dt.month/10)
data['target'] = np.log(data.target)
data['urf_code_map'] = data['store_id']

data.head()

,date,store_id,sales,other_feature1,other_feature2,other_feature3,other_feature4,target,urf_code_map
0,2022-01-31,1,272,0.467442,0.390914,0.460528,0.277053,4.236943,1
1,2022-01-31,2,147,0.048557,0.885066,0.253241,0.000043,2.880159,2
2,2022-01-31,3,217,0.990189,0.231459,0.015933,0.022026,4.398780,3
3,2022-01-31,4,292,0.761858,0.770246,0.477086,0.028540,4.387031,4
4,2022-01-31,5,423,0.967657,0.768680,0.064304,0.481964,4.897456,5


In [3]:
data.shape

(240000, 9)

In [4]:
data.columns

Index(['date', 'store_id', 'sales', 'other_feature1', 'other_feature2',
       'other_feature3', 'other_feature4', 'target', 'urf_code_map'],
      dtype='object')

In [5]:
# список фичей для включения в модель
model_list = ['sales', 'other_feature1', 'other_feature2',
       'other_feature3', 'other_feature4']

In [6]:
from sklearn.metrics import mean_absolute_percentage_error as mape
# создать базовую модель

data_sample = data[data['date'].dt.year == 2022]
data_sample.shape

(120000, 9)

In [7]:
model_base = LGBMRegressor(max_depth= 3, n_jobs= -1)

In [8]:
model_base.fit(data_sample[model_list], data_sample['target'])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002964 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1268
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 5
[LightGBM] [Info] Start training from score 5.858879
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMRegressor(max_depth=3, n_jobs=-1)

In [9]:
mape(data_sample['target'], np.exp(model_base.predict(data_sample[model_list])))

65.29332306326525

In [10]:
mape(data['target'], np.exp(model_base.predict(data[model_list])))

63.10098190667027

In [11]:
with open ("model_base.pkl", 'wb') as model_alpha:
    pickle.dump(model_base, model_alpha)

In [12]:
list_stores = data.store_id.unique().tolist()

In [13]:
random.seed(0)
subset_size = 2000
oos_list = random.sample(list_stores, subset_size)

In [14]:
data['dt'] = data.date

In [15]:
new_exp = AddTrain(df_ = data, 
                   model_path='model_base.pkl', 
                   train_end = '2022-10-31', 
                   oot_dates= ['2022-11-30', '2022-12-31'],
                   vsp_test = oos_list,
                   used_features = model_list)

In [16]:
new_exp.scoring_constant_model()

2022-11-30    10000
2022-12-31    10000
2023-01-31    10000
2023-02-28    10000
2023-03-31    10000
2023-04-30    10000
2023-05-31    10000
2023-06-30    10000
2023-07-31    10000
2023-08-31    10000
2023-09-30    10000
2023-10-31    10000
2023-11-30    10000
2023-12-31    10000
Name: dt, dtype: int64


In [17]:
new_exp.scoring_update_model()

*-*-*-*-*-*-*-*-*-*- start split train/test/oot *-*-*-*-*-*-*-*-*-*-
step_1


,dt,cnt_train,cnt_oos,cnt_oot
0,2022-01-31,8000.0,2000.0,NaN
1,2022-02-28,8000.0,2000.0,NaN
2,2022-03-31,8000.0,2000.0,NaN
3,2022-04-30,8000.0,2000.0,NaN
4,2022-05-31,8000.0,2000.0,NaN
5,2022-06-30,8000.0,2000.0,NaN
6,2022-07-31,8000.0,2000.0,NaN
7,2022-08-31,8000.0,2000.0,NaN
8,2022-09-30,8000.0,2000.0,NaN
9,2022-10-31,8000.0,2000.0,NaN


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002435 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1268
[LightGBM] [Info] Number of data points in the train set: 96000, number of used features: 5
[LightGBM] [Info] Start training from score 1.755533
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
*-*-*-*-*-*-*-*-*-*- start split train/test/oot *-*-*-*-*-*-*-*-*-*-
s

,dt,cnt_train,cnt_oos,cnt_oot
0,2022-01-31,8000.0,2000.0,NaN
1,2022-02-28,8000.0,2000.0,NaN
2,2022-03-31,8000.0,2000.0,NaN
3,2022-04-30,8000.0,2000.0,NaN
4,2022-05-31,8000.0,2000.0,NaN
5,2022-06-30,8000.0,2000.0,NaN
6,2022-07-31,8000.0,2000.0,NaN
7,2022-08-31,8000.0,2000.0,NaN
8,2022-09-30,8000.0,2000.0,NaN
9,2022-10-31,8000.0,2000.0,NaN


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007022 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1268
[LightGBM] [Info] Number of data points in the train set: 104000, number of used features: 5
[LightGBM] [Info] Start training from score 1.737327
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
*-*-*-*-*-*-*-*-*-*- start split train/test/oot *-*-*-*-*-

...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007897 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1269
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 5
[LightGBM] [Info] Start training from score 1.760725
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
*-*-*-*-*-*-*-*-*-*- start split train/test/oot *-*-*-*-*-*-*-*-*-*-
step_10


,dt,cnt_train,cnt_oos,cnt_oot
0,2022-01-31,8000.0,2000.0,NaN
1,2022-02-28,8000.0,2000.0,NaN
2,2022-03-31,8000.0,2000.0,NaN
3,2022-04-30,8000.0,2000.0,NaN
4,2022-05-31,8000.0,2000.0,NaN
5,2022-06-30,8000.0,2000.0,NaN
6,2022-07-31,8000.0,2000.0,NaN
7,2022-08-31,8000.0,2000.0,NaN
8,2022-09-30,8000.0,2000.0,NaN
9,2022-10-31,8000.0,2000.0,NaN


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007052 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1269
[LightGBM] [Info] Number of data points in the train set: 168000, number of used features: 5
[LightGBM] [Info] Start training from score 1.767997
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
*-*-*-*-*-*-*-*-*-*- start split train/test/oot *-*-*-*-*-*-*-*-*-*-
step_11


,dt,cnt_train,cnt_oos,cnt_oot
0,2022-01-31,8000.0,2000.0,NaN
1,2022-02-28,8000.0,2000.0,NaN
2,2022-03-31,8000.0,2000.0,NaN
3,2022-04-30,8000.0,2000.0,NaN
4,2022-05-31,8000.0,2000.0,NaN
5,2022-06-30,8000.0,2000.0,NaN
6,2022-07-31,8000.0,2000.0,NaN
7,2022-08-31,8000.0,2000.0,NaN
8,2022-09-30,8000.0,2000.0,NaN
9,2022-10-31,8000.0,2000.0,NaN


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019425 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1269
[LightGBM] [Info] Number of data points in the train set: 176000, number of used features: 5
[LightGBM] [Info] Start training from score 1.775316
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [19]:
new_exp.final_report()

,dt,const_mape_oot,const_macro_oot,const_macro_fact,const_ape_macro,update_w_mape_oot,update_w_macro_oot,update_w_macro_fact,update_w_ape_macro,diff
0,2022-11-30,56.89,3926908.66,65825.370878,5865.65,NaN,NaN,NaN,NaN,NaN
1,2022-12-31,56.22,3932108.33,66697.824008,5795.41,NaN,NaN,NaN,NaN,NaN
2,2023-01-31,81.26,3932195.80,45967.824128,8454.24,NaN,NaN,NaN,NaN,NaN
3,2023-02-28,70.73,3930188.83,52880.715215,7332.18,-0.10,58054.39,52880.715215,9.78,-100.14
4,2023-03-31,65.94,3944112.44,56931.946152,6827.77,-0.01,57018.90,56931.946152,0.15,-100.02
5,2023-04-30,62.79,3943077.34,59831.190683,6490.34,-0.05,56730.03,59831.190683,-5.18,-100.08
6,2023-05-31,60.36,3929691.71,62010.257106,6237.16,-0.09,56686.04,62010.257106,-8.59,-100.15
7,2023-06-30,58.91,3947557.40,63921.545762,6075.63,-0.11,56966.44,63921.545762,-10.88,-100.19
8,2023-07-31,57.92,3979052.51,65517.107508,5973.30,-0.13,57310.78,65517.107508,-12.53,-100.22
9,2023-08-31,56.51,3958536.26,66780.676825,5827.67,-0.14,57587.38,66780.676825,-13.77,-100.25
